<a href="https://colab.research.google.com/github/oz-e/applied-ml/blob/main/CuPL_pets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Change the path if necessary
datasets_path = 'datasets'

import sys
import os

# Deploy AML code in colab
if 'google.colab' in sys.modules:
  !wget -q https://raw.githubusercontent.com/tsunrise/colab-github/main/colab_github.py
  import colab_github
  colab_github.github_auth(persistent_key=False)

  # Remember to follow the instructions to add the SSH key to your account in order to clone private repo
  if os.path.exists('/content/applied-ml/'):
    %cd /content/applied-ml/
  else:
    !git clone git@github.com:oz-e/applied-ml.git
    if not os.path.exists('/content/applied-ml/'):
      raise Exception('Please follow the instructions to add the SSH key to your account in order to clone private repo')
    %cd /content/applied-ml/

  # Install any other requirements (to be converted to requirements.txt)
  !pip install openai-clip

Looks that a private key is already created. If you have already push it to github, no action required.
 Otherwise, Please go to https://github.com/settings/ssh/new to upload the following key: 
ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIGBiupHPwjOzra71Dyk4fM71hMWKnim4Tf23ohVZgoS6 root@8e337fcc3bba

Please use SSH method to clone repo.
Cloning into 'applied-ml'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 88 (delta 46), reused 18 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (88/88), 1.92 MiB | 5.92 MiB/s, done.
Resolving deltas: 100% (46/46), done.
/content/applied-ml
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 24.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.9 MB/s eta 0:00:00
  Created wheel for openai-clip: filename=openai_clip-1.0.1-py3-none-any.whl size=1368605 sha256=eb35af36733eae07f

In [3]:
import os

if not os.path.exists("CuPL"):
  !git clone https://github.com/sarahpratt/CuPL.git

Cloning into 'CuPL'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 56 (delta 6), reused 52 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (56/56), 4.68 MiB | 15.26 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [4]:
import numpy as np
import torch
import clip
from pkg_resources import packaging
from CuPL.imagenet_prompts.standard_image_prompts import imagenet_templates
import pdb
from collections import defaultdict
from PIL import Image
import PIL
import json
from tqdm import tqdm

In [5]:
model, preprocess = clip.load("ViT-B/16")

100%|████████████████████████████████████████| 335M/335M [00:02<00:00, 121MiB/s]


In [6]:
import aml.datasets

all_images = aml.datasets.OxfordIIITPet(datasets_path, split='test', transform=preprocess)
loader = torch.utils.data.DataLoader(all_images, batch_size=512, num_workers=8)

100%|██████████| 792M/792M [00:31<00:00, 25.2MB/s]
100%|██████████| 19.2M/19.2M [00:01<00:00, 12.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1501r8Ber4nNKvmlFVQZ8SeUHTcdTTEqs
To: /content/applied-ml/datasets/oxford-iiit-pet/split.json
100%|██████████| 724k/724k [00:00<00:00, 109MB/s]
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [7]:
PATH_TO_PROMPTS = './CuPL/all_prompts/full_prompts/pets_prompts_full.json'

with open(PATH_TO_PROMPTS) as f:
  gpt3_prompts = json.load(f)

In [8]:
special_class_mapping = {
}

In [9]:
# For bridging classsname in dataset with name in CuLP prompts
new_gpt3_prompts = {}

for key, value in gpt3_prompts.items():
  new_key = special_class_mapping.get(key, key)
  new_gpt3_prompts[new_key.replace(' ', '_').lower()] = value

In [10]:
def zeroshot_classifier(classnames, templates):
	with torch.no_grad():
		zeroshot_weights = []
		for classname in tqdm(classnames):
			texts = [template.format(classname) for template in templates] #format with class
			texts = clip.tokenize(texts).cuda() #tokenize
			class_embeddings = model.encode_text(texts) #embed with text encoder
			class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
			class_embedding = class_embeddings.mean(dim=0)
			class_embedding /= class_embedding.norm()
			zeroshot_weights.append(class_embedding)
		zeroshot_weights = torch.stack(zeroshot_weights, dim=1).cuda()
	return zeroshot_weights

In [11]:
def zeroshot_classifier_gpt(classnames, templates, use_both):
	with torch.no_grad():
		zeroshot_weights = []
		for classname in tqdm(classnames):
			if use_both:
				texts = [template.format(classname) for template in templates]
			else:
				texts = []

			for t in new_gpt3_prompts[classname]:
				texts.append(t)
			texts = clip.tokenize(texts, truncate=True).cuda() #tokenize
			class_embeddings = model.encode_text(texts) #embed with text encoder
			class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
			class_embedding = class_embeddings.mean(dim=0)
			class_embedding /= class_embedding.norm()
			zeroshot_weights.append(class_embedding)

		zeroshot_weights = torch.stack(zeroshot_weights, dim=1).cuda()
	return zeroshot_weights

In [12]:
print("\nCreating standard text embeddings...")
zeroshot_weights_base = zeroshot_classifier(all_images.classnames, imagenet_templates)
print("Done.\n")

print("Creating CuPL text embeddings...")
zeroshot_weights_cupl = zeroshot_classifier_gpt(all_images.classnames, imagenet_templates, False)
print("Done.\n")

print("Creating combined text embeddings...")
zeroshot_weights_gpt_both = zeroshot_classifier_gpt(all_images.classnames, imagenet_templates, True)
print("Done.\n")


Creating standard text embeddings...


100%|██████████| 37/37 [00:02<00:00, 13.69it/s]


Done.

Creating CuPL text embeddings...


100%|██████████| 37/37 [00:00<00:00, 66.83it/s]


Done.

Creating combined text embeddings...


100%|██████████| 37/37 [00:02<00:00, 14.58it/s]

Done.



In [13]:
total = 0.
correct_base = 0.
correct_cupl = 0.
correct_both = 0.

print("Classifying ImageNet...")

with torch.no_grad():

	for i, (images, target) in enumerate(tqdm(loader)):
		images = images.cuda()
		target = target.cuda()

		# predict
		image_features = model.encode_image(images)
		image_features /= image_features.norm(dim=-1, keepdim=True)

		logits_base = image_features @ zeroshot_weights_base
		logits_cupl = image_features @ zeroshot_weights_cupl
		logits_both = image_features @ zeroshot_weights_gpt_both

		pred_base = torch.argmax(logits_base, dim =1)
		pred_cupl = torch.argmax(logits_cupl, dim =1)
		pred_both = torch.argmax(logits_both, dim =1)

		for j in range(len(target)):
			total += 1.
			if pred_base[j] == target[j]:
				correct_base += 1.
			if pred_cupl[j] == target[j]:
				correct_cupl += 1.
			if pred_both[j] == target[j]:
				correct_both += 1.

Classifying ImageNet...


100%|██████████| 8/8 [00:36<00:00,  4.58s/it]


In [14]:
print()
top1 = (correct_base / total) * 100
print(f"Top-1 accuracy standard: {top1:.2f}")

top1 = (correct_cupl / total) * 100
print(f"Top-1 accuracy CuPL: {top1:.2f}")

top1 = (correct_both / total) * 100
print(f"Top-1 accuracy both: {top1:.2f}")


Top-1 accuracy standard: 87.05
Top-1 accuracy CuPL: 91.22
Top-1 accuracy both: 88.53
